### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
# filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [21]:
# Create an empty list to store the reviews
reviews_list = []
# loop through pages 0-19
page = 0
# API results show 10 articles at a time
# any number of calls more than 5 getting policy violation error so limiting to 5
while page <= 4: 
    # create query with a page number
    # Make a "GET" request and retrieve the JSON
    updated_url_with_page = f"{query_url}&page={page}"
    response = requests.get(updated_url_with_page)
    reviews = response.json()    

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in reviews["response"]["docs"]:
            reviews_list.append(doc)
        # Print the page that was just retrieved
        print(f"Found articles for the page {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No articles found in the page {page}")
    page = page + 1



Found articles for the page 0
Found articles for the page 1
Found articles for the page 2
Found articles for the page 3
Found articles for the page 4


In [22]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for reviews in reviews_list:
    print(json.dumps(reviews, indent=4))


{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [24]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'fir

In [25]:
title_list = []
for reviews in reviews_list:
    # Extract the title from the "headline.main" column and
    title = reviews["headline"]["main"]
    start = title.index('\u2018')
    end = title.index('\u2019',start+1)
    # Title is between unicode characters \u2018 and \u2019. 
    substring_tittle = title[start+1:end]
    # End string should include " Review" to avoid cutting title early
    # title = substring_tittle + " Review"
    title_list.append(substring_tittle)
# save it to a new column "title"
reviews_df["title"] = title_list
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",

In [27]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df.loc[:, "keywords"].astype(str)
for reviews in reviews_list:
    keywords_list = []
    keywords_list = reviews["keywords"]
    extract_keyword = extract_keywords(keywords_list)
    reviews_df["keywords"] = extract_keyword
reviews_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+

In [28]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
tittle = reviews_df["title"]
tittle_list = tittle.to_list()
tittle_list

['The Attachment Diaries',
 'What',
 'You Can Live Forever',
 'A Tourist',
 'Other People',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie']

### Access The Movie Database API

In [29]:
# Prepare The Movie Database query
tmdb_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [30]:
# Create an empty list to store the results
tmdb_movies_list = []
movies_data = {}
tmdb_movie_detail_url = "https://api.themoviedb.org/3/movie/"

# Loop through the titles
for movie_title in tittle_list:
    movie_data = {}

    # Check if we need to sleep before making a request
    # Add 1 to the request counter
    
    # Perform a "GET" request for The Movie Database
    response_data = requests.get(tmdb_url + movie_title + tmdb_key_string)
    mov_title_data = response_data.json()
    movie_data['title'] = movie_title

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # loop through the movie tittle ["results"] to get the movie id
        for doc in mov_title_data["results"]:
            movie_id = doc['id']
            genre_list = []
            spoken_languages_list = []
            production_countries_list = []
            # Get movie id
            # Make a request for a the full movie details
            # Execute "GET" request with url
            tmdb_movie_detail_by_id = requests.get(tmdb_movie_detail_url + str(doc['id']) + "?" + tmdb_key_string)
            tmdb_movie_detail = tmdb_movie_detail_by_id.json()
            # Add the relevant data to a dictionary and
            # Extract the genre names into a list
            for genre in tmdb_movie_detail["genres"]:
                genre_list.append(genre['name'])
            movie_data['genres'] = genre_list
            # Extract the spoken_languages' English name into a list
            for spoken_languages in tmdb_movie_detail["spoken_languages"]:
                spoken_languages_list.append(spoken_languages['english_name'])
            movie_data['spoken_languages'] = spoken_languages_list
            # Extract the production_countries' name into a list
            for production_countries in tmdb_movie_detail["production_countries"]:
                production_countries_list.append(production_countries['name'])
            movie_data['production_countries'] = production_countries_list
            # append it to the tmdb_movies_list list 
            tmdb_movies_list.append(movie_data)
        # Print the page that was just retrieved
        # Print out the title that was found
        print(f"Movie Found :", movie_title)
    except:
        # Print the page number that had no results then break from the loop
        print(f"Movie Ids Not Found")
print(f"Final Movie dictionary:", tmdb_movies_list)


Movie Found : The Attachment Diaries
Movie Found : What
Movie Found : You Can Live Forever
Movie Found : A Tourist
Movie Found : Other People
Movie Found : One True Loves
Movie Found : The Lost Weekend: A Love Story
Movie Found : A Thousand and One
Movie Found : Your Place or Mine
Movie Found : Love in the Time of Fentanyl
Movie Found : Pamela, a Love Story
Movie Found : In From the Side
Movie Found : After Love
Movie Found : Alcarràs
Movie Found : Nelly & Nadine
Movie Found : Lady Chatterley
Movie Found : The Sound of Christmas
Movie Found : The Inspection
Movie Found : Bones and All
Movie Found : My Policeman
Movie Found : About Fate
Movie Found : Waiting for Bojangles
Movie Found : I Love My Dad
Movie Found : A Love Song
Movie Found : Alone Together
Movie Found : Art of Love
Movie Found : The Wheel
Movie Found : Thor: Love and Thunder
Movie Found : Both Sides of the Blade
Movie Found : Fire of Love
Movie Found : Love & Gelato
Movie Found : Stay Prayed Up
Movie Found : Benediction
Mo

In [31]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
json.dumps(tmdb_movies_list, indent=4)

'[\n    {\n        "title": "The Attachment Diaries",\n        "genres": [\n            "Drama",\n            "Mystery",\n            "Thriller",\n            "Horror"\n        ],\n        "spoken_languages": [\n            "Spanish"\n        ],\n        "production_countries": [\n            "Argentina"\n        ]\n    },\n    {\n        "title": "What",\n        "genres": [\n            "Comedy",\n            "Drama"\n        ],\n        "spoken_languages": [\n            "German"\n        ],\n        "production_countries": [\n            "Germany"\n        ]\n    },\n    {\n        "title": "What",\n        "genres": [\n            "Comedy",\n            "Drama"\n        ],\n        "spoken_languages": [\n            "German"\n        ],\n        "production_countries": [\n            "Germany"\n        ]\n    },\n    {\n        "title": "What",\n        "genres": [\n            "Comedy",\n            "Drama"\n        ],\n        "spoken_languages": [\n            "German"\n       

In [32]:
# Convert the results to a DataFrame
all_results_df = pd.json_normalize(tmdb_movies_list)
all_results_df


,title,genres,spoken_languages,production_countries
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What,"[Comedy, Drama]",[German],[Germany]
2,What,"[Comedy, Drama]",[German],[Germany]
3,What,"[Comedy, Drama]",[German],[Germany]
4,What,"[Comedy, Drama]",[German],[Germany]
...,...,...,...,...
499,West Side Story,[Music],[],[]
500,West Side Story,[Music],[],[]
501,West Side Story,[Music],[],[]
502,West Side Story,[Music],[],[]


### Merge and Clean the Data for Export

In [35]:
# Merge the New York Times reviews and TMDB DataFrames on title
merge_df = pd.merge(reviews_df, all_results_df, on="title", how="outer")
merge_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"[Comedy, Drama]",[German],[Germany]
2,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"[Comedy, Drama]",[German],[Germany]
3,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"[Comedy, Drama]",[German],[Germany]
4,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"[Comedy, Drama]",[German],[Germany]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,https://www.nytimes.com/2021/12/08/movies/west...,"Steven Spielberg rediscovers the breathing, tr...",The New York Times,subject: Documentary Films and Programs;subjec...,2021-12-08T14:55:30+0000,1316,"‘West Side Story’ Review: In Love and War, 195...",Critic’s Pick,None,"Meeting a Girl Named Maria, Again",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,West Side Story,[Music],[],[]
501,https://www.nytimes.com/2021/12/08/movies/west...,"Steven Spielberg rediscovers the breathing, tr...",The New York Times,subject: Documentary Films and Programs;subjec...,2021-12-08T14:55:30+0000,1316,"‘West Side Story’ Review: In Love and War, 195...",Critic’s Pick,None,"Meeting a Girl Named Maria, Again",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,West Side Story,[Music],[],[]
502,https://www.nytimes.com/2021/12/08/movies/west...,"Steven Spielberg rediscovers the breathing, tr...",The New York Times,subject: Documentary Films and Programs;subjec...,2021-12-08T14:55:30+0000,1316,"‘West Side Story’ Review: In Love and War, 195...",Critic’s Pick,None,"Meeting a Girl Named Maria, Again",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,West Side Story,[Music],[],[]
503,https://www.nytimes.com/2021/12/08/movies/west...,"Steven Spielberg rediscovers the breathing, tr...",The New York Times,subject: Documentary Films and Programs;subjec...,2021-12-08T14:55:

In [41]:
# Remove list brackets and quotation marks on the columns containing lists

merge_df['spoken_languages'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in merge_df['spoken_languages']])
merge_df['genres'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in merge_df['genres']])
merge_df['production_countries'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in merge_df['production_countries']])

# Create a list of characters to remove
# Loop through the list of columns to fix
    # Convert the column to type 'str'
    # Loop through characters to remove

# Display the fixed DataFrame
merge_df.head(10)



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
2,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
3,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
4,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
5,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
6,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
7,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
8,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
9,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany


In [37]:
# Drop "byline.person" column
del merge_df['byline.person']
merge_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
2,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
3,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
4,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,https://www.nytimes.com/2021/12/08/movies/west...,"Steven Spielberg rediscovers the breathing, tr...",The New York Times,subject: Documentary Films and Programs;subjec...,2021-12-08T14:55:30+0000,1316,"‘West Side Story’ Review: In Love and War, 195...",Critic’s Pick,None,"Meeting a Girl Named Maria, Again",None,None,None,By A.O. Scott,None,West Side Story,Music,,
501,https://www.nytimes.com/2021/12/08/movies/west...,"Steven Spielberg rediscovers the breathing, tr...",The New York Times,subject: Documentary Films and Programs;subjec...,2021-12-08T14:55:30+0000,1316,"‘West Side Story’ Review: In Love and War, 195...",Critic’s Pick,None,"Meeting a Girl Named Maria, Again",None,None,None,By A.O. Scott,None,West Side Story,Music,,
502,https://www.nytimes.com/2021/12/08/movies/west...,"Steven Spielberg rediscovers the breathing, tr...",The New York Times,subject: Documentary Films and Programs;subjec...,2021-12-08T14:55:30+0000,1316,"‘West Side Story’ Review: In Love and War, 195...",Critic’s Pick,None,"Meeting a Girl Named Maria, Again",None,None,None,By A.O. Scott,None,West Side Story,Music,,
503,https://www.nytimes.com/2021/12/08/movies/west...,"Steven Spielberg rediscovers the breathing, tr...",The New York Times,subject: Documentary Films and Programs;subjec...,2021-12-08T14:55:30+0000,1316,"‘West Side Story’ Review: In Love and War, 195...",Critic’s Pick,None,"Meeting a Girl Named Maria, Again",None,None,None,By A.O. Scott,None,West Side Story,Music,,


In [38]:
# Delete duplicate rows and reset index
merge_df_reinded_df = merge_df["title"].drop_duplicates().reset_index(drop=True)
merge_df_reinded_df

0             The Attachment Diaries
1                               What
2               You Can Live Forever
3                          A Tourist
4                       Other People
5                     One True Loves
6     The Lost Weekend: A Love Story
7                 A Thousand and One
8                 Your Place or Mine
9       Love in the Time of Fentanyl
10              Pamela, a Love Story
11                  In From the Side
12                        After Love
13                          Alcarràs
14                    Nelly & Nadine
15                   Lady Chatterley
16            The Sound of Christmas
17                    The Inspection
18                     Bones and All
19                      My Policeman
20                        About Fate
21             Waiting for Bojangles
22                     I Love My Dad
23                       A Love Song
24                    Alone Together
25                       Art of Love
26                         The Wheel
2

In [39]:
# Export data to CSV without the index
merge_df.to_csv("movies_list.csv", index=False)